Functions that calculate time of inference of a schnet model...

can construct a set of ordered moelcules (straight chain alcohols) and then statistical computation of inference for each

In [41]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os


starting_molfilepath = 'utils/data/timing/methanol.mol'
carbon_number = 2
molecule = Chem.MolFromMolFile(starting_molfilepath)
molecule = Chem.MolToMolBlock(molecule)
number_atoms = 9

c_line = '    0.0000   -0.0000   -0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0  \n'

save_xyzfilepath = 'utils/data/timing/strchnalcs.xyz'
xyz_file = open(save_xyzfilepath,mode='a')

for i in range(50):

    if carbon_number < 9:
        bond_line = '  '+str(carbon_number)+'  '+str(carbon_number+1) + '  1  0' 
    if carbon_number  == 9:
        bond_line = ' '+str(carbon_number)+'  '+str(carbon_number+1) + '  1  0'    
    if carbon_number > 9:
        bond_line = ' '+str(carbon_number)+' '+str(carbon_number+1) + '  1  0'     
    
    new_molecule = ''
    for line_index, line in enumerate(molecule.split('\n')):
        if line_index == 3:
            if carbon_number < 9:
                line = line.replace(line,'  '+str(carbon_number+1)+'  '+str(carbon_number)+line[6:40])
            if carbon_number == 9:
                line = line.replace(line,' '+str(carbon_number+1)+'  '+str(carbon_number)+line[6:40])
            if carbon_number > 9:
                line = line.replace(line,' '+str(carbon_number+1)+' '+str(carbon_number)+line[6:40])   
        if line_index == 3 + carbon_number+1: 
            line = c_line + bond_line + '\n' + line
        new_molecule = new_molecule + line +'\n'
    
    temp_molfilepath = 'utils/data/timing/temp/strchnalcs%s.mol' %(i)
    temp_molfile = open(temp_molfilepath,mode='w')

    molecule = new_molecule
    carbon_number = carbon_number+1

    new_molecule = Chem.MolFromMolBlock(new_molecule)
    new_molecule = Chem.AddHs(new_molecule,addCoords=True)
    AllChem.EmbedMolecule(new_molecule)
    #AllChem.MMFFOptimizeMolecule(new_molecule)
    new_molecule = Chem.MolToMolBlock(new_molecule)

    temp_molfile.write(new_molecule)
    temp_molfile.close()

    temp_xyzfilepath = temp_molfilepath.replace('mol','xyz')
    os.system('obabel ' + temp_molfilepath + ' -O ' + temp_xyzfilepath + ' >/dev/null 2>&1') 

    temp_xyzfile = open(temp_xyzfilepath,mode='r')

    for line_index, line in enumerate(temp_xyzfile.read().split('\n')):
        if line_index == 1:
            line = line.replace(line,'0.0000')

        if line_index < number_atoms+2:
            xyz_file.write(line + ' \n')
        if line_index == number_atoms+2:
            xyz_file.write(line[0:46])   

    number_atoms = number_atoms + 3
    temp_xyzfile.close()

    os.system('rm ' + temp_molfilepath)
    os.system('rm ' + temp_xyzfilepath)

xyz_file.close()



Time inference statistics

In [1]:
from utils import utils_schnet
import time

start = time.time()

for i in range(10000):
#dataset
    #dataset_filepath = 'chiral-1.db'
    #dataset_filepath = 'fgtransform/data/model2/1alcseths/trans.db'
    qm9 = False
    dataset_filepath = 'utils/data/timing/strchnalcs.db'

    #test molecule
    idx=5

    #trainedmodel
    model_filepath = '../data/trainedmodels/model2/best_model'
    #load it with torch, and use CPU if you trained with CPU

    pred = utils_schnet.run_model(dataset_filepath,model_filepath,idx,qm9)

end = time.time()

print(pred)
print((end - start)/10000)

/home/amerelsamman/anaconda3/envs/embs/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.008400063157081604
